In [1]:
import os
import pandas as pd
#from __future__ import unicode_literals
import sys  
import re
import fileinput
import codecs
import numpy as np
import re
import pickle
import math
import itertools
import editdistance
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.translate import bleu_score
from nltk.tokenize import RegexpTokenizer

# reload(sys)  
# sys.setdefaultencoding('utf-8')

## Tools to parse bibliography references

In [25]:
bib_names = [u"литература",
            u"список литературы",
            u"цитируемая литература",
            u"список цитируемой литературы",
            u"список литературных источников",
            u"список использованных источников",
            u"список источников и литературы",
            u"список использованных источников и литературы",
            u"список рекомендуемой литературы",
            u"список использованной литературы",
            u"список публикаций",
            u"список публикаций по теме диссертации",
            u"список работ, опубликованных по теме диссертации",
            u"список основных публикаций по теме диссертации",
            u"список опубликованных работ",
            u"публикации по теме диссертации",
            u"публикации автора по теме диссертации",
            u"работы автора по теме диссертации",
            u"основные публикации по теме диссертации",
            u"библиография",
            u"библиографический список",
            u"ссылки на источники",
            u"список використаних джерел",
            u"збірник наукових праць",
            u"бібліографічний список",
            u"література",
            u"список літератури",
            u"список опублікованих праць за темою дисертації",
            u"статті у наукових фахових виданнях"] # добавить?

In [4]:
tokenizer = RegexpTokenizer(r'\w+')

def count_occurences_in_text(text, substring_list):
    count_list = []
    for substring in substring_list:
        count_list.append(len([m.start() for m in re.finditer(substring, text.lower())]))
    return count_list

In [5]:
def remove_hyphenation(text):
    # remove empty lines
    text = re.sub("\n\s*\n*", "\n", text)
    dehyphenated = re.sub("^\s|-\n|-\r|\s+$", '', text)
    deh = re.sub("[ ]–\n|[ ]–[ ]\n", " – ", dehyphenated)
    arr = deh.split('\n')
    res = ''
    for a in arr:
        if not a.endswith("."):
            res += a + ' '
        else:
            res += a + '\n'
    return res

In [6]:
def parse_bib_block(text):
    pattern = re.compile("(?:(?<!\d)\d{1,3}(?!\d))\. ?[^\n]* \d{4}") 
    num_occurrences = sum(count_occurences_in_text(text, bib_names))
    if num_occurrences == 1:
        count_dict = dict(zip(bib_names, count_occurences_in_text(text, bib_names)))
        for name, val in count_dict.items():
            if val == 1:
                cur_bibname = name
        #print (cur_bibname)
        index = text.lower().index(cur_bibname) + len(cur_bibname)
        bib_block = text[index:].strip()
        strings = bib_block.split("\n")
        for string in strings:
            if pattern.match(string):
                references.append(string) 
    elif num_occurrences == 0:
        references = []
    else:
        # Весьма так себе костыль
        references = []
        min_index = 10**7
        for substring in bib_names:
            if substring in text.lower():
                indices = [m.start() for m in re.finditer(substring, text.lower())]
                if indices and min(indices) < min_index:
                    min_index = min(indices)
        strings = text[min_index:].strip().split('\n')
        for string in strings:
            if pattern.match(string):
                references.append(string)  
    return references   

## Collecting bibliography references

In [6]:
data_path = "unlabeled"
paths = os.listdir(data_path)

In [17]:
df = pd.DataFrame(columns=['ref1', 'ref2'])

In [18]:
row = 0
for path in paths:    
    dir_name = os.path.join(data_path, path)
    with open(os.path.join(dir_name, 'description.txt')) as text:
        ideal_ref = text.readline()
    
    print ('ideal', ideal_ref)
    for root, dirs, files in os.walk(dir_name):
        for f in files:
            if not f.endswith('out.txt') and not f.endswith('out-.txt'):
                with open(os.path.join(root, f)) as txt:
                    try:
                        ttt = txt.read() 
                        deh = remove_hyphenation(ttt)
                        refs = parse_bib_block(deh)
                        for r in refs:
                            if len(r) > 10:
                                df.loc[row] = [ideal_ref, r]
                                row += 1
                    except UnicodeDecodeError:
                        continue

ideal Козырева О.А. СПЕЦИФИКА СОЗДАНИЯ ПЕДАГОГИЧЕСКИХ УСЛОВИЙ ВКЛЮЧЕНИЯ БУДУЩЕГО ПЕДАГОГА В НАУЧНО-ИССЛЕДОВАТЕЛЬСКУЮ РАБОТУ. Вестник Кемеровского государственного университета.  2015. № 2-3.  С. 63-67
ideal Полтерович В. Стратегии модернизации, институты и коалиции. Вопросы экономики.  2008. № 4.  С. 4-24
ideal Демьянов А.В., Котов А.Ю., Симбирцев А.С. Диагностическая ценность исследования уровней цитокинов в клинической практике. Цитокины и воспаление.  2003. Т. 2. № 3.  С. 20-35
ideal Баранов А.А., Щеплягина Л.А., Ильин А.Г., Кучма В.Р. СОСТОЯНИЕ ЗДОРОВЬЯ ДЕТЕЙ КАК ФАКТОР НАЦИОНАЛЬНОЙ БЕЗОПАСНОСТИ. Российский педиатрический журнал.  2005. № 2.  С. 4-8
ideal Козырева О.А. ТЕХНОЛОГИЯ СИСТЕМНО-ПЕДАГОГИЧЕСКОГО МОДЕЛИРОВАНИЯ И КАЧЕСТВО ФОРМИРОВАНИЯ КУЛЬТУРЫ САМОСТОЯТЕЛЬНОЙ РАБОТЫ ПЕДАГОГОВ: ТЕОРЕТИЧЕСКИЙ АСПЕКТ. European Social Science Journal.  2014. № 4-1 (43).  С. 136-142
ideal Полтерович В.М. Трансплантация экономических институтов. Экономическая наука современной России.  2001. № 3. 

In [24]:
df['cosine'] = df.apply(cosine, axis=1) 
df['levenshtein'] = df.apply(levenshtein, axis=1) 
df['jaccard'] = df.apply(jaccard, axis=1) 
df['bleu'] = df.apply(bleu, axis=1)
df['tfidf_cosine'] = df.apply(tfidf_cosine, axis=1)
df['simhash'] = df.apply(simhash_feature, axis=1)
df['ft'] = df.apply(weighted_fasttext, axis=1)

In [27]:
df = df.dropna()

In [21]:
df.to_csv('unlabeled.csv', index=False)

## Predict labels

In [35]:
with open('rf.pkl', 'rb') as fid:
    rfc = pickle.load(fid)

In [6]:
X = df.drop(['ref1', 'ref2'], axis=1)

In [36]:
df['predicted'] = rfc.predict(X)

In [34]:
df.to_csv('unlabeled_predicted.csv', index=False)

## Analyse & count errors

In [37]:
positive = df[df.predicted==1]
negative = df[df.predicted==0]

In [25]:
for ind, row in positive.iterrows():
    print (ind, row.ref1, '\n', row.ref2, '\n')

3701 Полтерович В. Стратегии модернизации, институты и коалиции. Вопросы экономики.  2008. № 4.  С. 4-24 
 8. Полтерович В. Стратегии модернизации, институты и коалиции / В. Полтерович // Вопросы экономики. – 2008. – № 4. – С. 4-24. 

3836 Полтерович В. Стратегии модернизации, институты и коалиции. Вопросы экономики.  2008. № 4.  С. 4-24 
 177. Полтерович В.М.. Стратегии модернизации, институты и коалиции // Вопросы экономики, №4, 2008. С. 4–24. 

3905 Полтерович В. Стратегии модернизации, институты и коалиции. Вопросы экономики.  2008. № 4.  С. 4-24 
 275. Cohen, S.B. Geography and Politics in a Divided World. [Текст] / S.B. Cohen. – London: Methuen, 1964. 

3916 Полтерович В. Стратегии модернизации, институты и коалиции. Вопросы экономики.  2008. № 4.  С. 4-24 
 292. Hansen N. The Economic Development of Border Regions // Growth and Change, 1977, Vol. 8, pp. 2-8. 

3917 Полтерович В. Стратегии модернизации, институты и коалиции. Вопросы экономики.  2008. № 4.  С. 4-24 
 293. Hansen N

In [28]:
for ind, row in negative.sample(1000).iterrows():
    print (ind, row.ref1, '\n', row.ref2, '\n')

95436 Болдырев. Н.Н. Концептуальное пространство когнитивной лингвистики. Вопросы когнитивной лингвистики.  2004. № 1.  С. 18-36 
 2.	 Бенвенист Э. Общая лингвистика. М., 1974. 

76254 Байденко В. Компетенции в профессиональном образовании (К освоению компетентностного подхода). Высшее образование в России.  2004. № 11.  С. 3-13 
 4. Педагогика: учеб. пособие для студентов пед.  ин-тов  /  Ю.К.Бабанский,  В.А.  Сластенин, Н.А.Сорокин и др.; под ред. Ю.К.Бабанского. – М.; Просвещение, 1988. – 2-е изд. доп. и перераб. – С.48–49. 

53535 Демьянков В.З. Политический дискурс как предмет политологической филологии. Политическая наука.  2002. № 3.  С. 31-44 
 64. Григорьева, О. Н. Политический театр современной России (взгляд филолога) О. Н. Григорьева // Интернет-журнал «Полемика». – Публикации Irex-Russia, 2001. – № 9. 

79864 Андреев A. Знания или компетенции?. Высшее образование в России.  2005. № 2.  С. 3-11 
 209. Толлингерова, Д.А. Анализ когнитивного состава задач с помощью вычислител